In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
text_pos = pd.read_csv('positive.csv',header=None, sep=";")[3][0:2000]
text_neg = pd.read_csv('negative.csv',header=None, sep=";")[3][0:2000]

In [3]:
print(text_pos.head())
print(text_neg.head())

0    @first_timee хоть я и школота, но поверь, у на...
1    Да, все-таки он немного похож на него. Но мой ...
2    RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3    RT @digger2912: "Кто то в углу сидит и погибае...
4    @irina_dyshkant Вот что значит страшилка :D\nН...
Name: 3, dtype: object
0    на работе был полный пиддес :| и так каждое за...
1    Коллеги сидят рубятся в Urban terror, а я из-з...
2    @elina_4post как говорят обещаного три года жд...
3    Желаю хорошего полёта и удачной посадки,я буду...
4    Обновил за каким-то лешим surf, теперь не рабо...
Name: 3, dtype: object


In [4]:
text_pos = text_pos.to_frame()
text_neg = text_neg.to_frame()

In [5]:
text_pos[4] = 1
text_neg[4] = 0

In [6]:
text_pos = text_pos.rename(columns={3: 'Data', 4: 'Label'})
text_neg = text_neg.rename(columns={3: 'Data', 4: 'Label'})


In [7]:
text_all = pd.concat([text_pos, text_neg])

In [8]:
text_all = text_all.reset_index(drop=True)

In [9]:
text_all = text_all.sample(frac=1).reset_index(drop=True)

In [10]:
text_all

,Data,Label
0,"жаль, что не написал записки для #helloday зар...",0
1,племянница 14ти лет набила татухи.. куда катит...,0
2,Хоть я года три назад и писал микростатью на х...,0
3,"Совсем скоро НАШ концерт!!! 10 декабря!""Бочка""...",1
4,@lionheartinside иди против них!бунтуй!ложись ...,1
...,...,...
3995,я сметанный колобок:)\nахахааххаах\nсегодня вс...,1
3996,"пошли гулять, стало плохо, подруку доводили до...",0
3997,RT @30Chemical_Day: @Echelonidiot30 пищально( ...,0
3998,"Только что рассказывал доклад по истории,про Д...",1


In [11]:
text, text_test = train_test_split(text_all, test_size=0.2)

In [12]:
print(len(text))
print(len(text_test))

3200
800


In [13]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')

Using cache found in /home/andreiliphd/.cache/torch/hub/pytorch_fairseq_master


loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz from cache at /home/andreiliphd/.cache/torch/pytorch_fairseq/83e3a689e28e5e4696ecb0bbb05a77355444a5c8a3437e0f736d8a564e80035e.c687083d14776c1979f3f71654febb42f2bb3d9a94ff7ebdfe1ac6748dba89d2
| dictionary: 50264 types


In [14]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, text):
        super(TextDataset, self).__init__()
        self.text = text
        
    def __getitem__(self, index):
        encoded = roberta.encode(self.text.iloc[index].Data)
        encoded = torch.nn.functional.pad(encoded, pad=(0,400 - encoded.shape[0]), mode='constant', value=2)
        return encoded, self.text.iloc[index].Label
    
    def __len__(self):
        return len(self.text)
    


In [15]:
dl_text = torch.utils.data.DataLoader(TextDataset(text),batch_size=64, shuffle=True)
dl_text_test = torch.utils.data.DataLoader(TextDataset(text_test),batch_size=64, shuffle=True)

In [16]:
class RoBERTaClassifier(torch.nn.Module):
    def __init__(self):
        super(RoBERTaClassifier, self).__init__()
        self.fc1 = torch.nn.Linear(1024, 128)
        self.fc2 = torch.nn.Linear(51200, 128)
        self.fc3 = torch.nn.Linear(128, 2)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        x = roberta.extract_features(x)        
        x = torch.nn.functional.relu(self.fc1(x))        
        x = torch.nn.functional.relu(self.fc2(x.view(-1, 51200)))
        x = self.fc3(x)
        return x

In [17]:
model = RoBERTaClassifier()

In [18]:
for par in roberta.parameters():
    par.requires_grad = False

In [19]:
model.cuda()
roberta.cuda()

RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          )
          (1): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=1024, out_features=1024, bias=Tru

In [20]:
criterion = torch.nn.CrossEntropyLoss()

In [21]:
# paramters = list(model.parameters()) + list(roberta.parameters())
parameters = model.parameters()
optimizer = torch.optim.Adam(parameters, lr=0.0005)

In [22]:
for epoch in range(30):
    loss_acc = 0
    model.train()
    for data,label in dl_text:
        data = data.cuda()
        label = label.cuda()
        output = model(data)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        print(loss.item())
        loss_acc +=loss.item()
        optimizer.step()
    print('Epoch: ', epoch, ', Loss: ', loss_acc/(3200/64))
    
    total_correct = 0
    total = 0
    model.eval()
    for data,label in dl_text:
        data, label = data.cuda(), label.cuda()
        output = model(data)
        loss = criterion(output, label)
        max_arg_output = torch.argmax(output, dim=1)
        total_correct += int(torch.sum(max_arg_output == label))
        total += data.shape[0]
    print('Training accuracy: {:.0%}'.format(total_correct/total))

    
    total_correct = 0
    total = 0
    model.eval()
    for data,label in dl_text_test:
        data, label = data.cuda(), label.cuda()
        output = model(data)
        loss = criterion(output, label)
        max_arg_output = torch.argmax(output, dim=1)
        total_correct += int(torch.sum(max_arg_output == label))
        total += data.shape[0]
    print('Test accuracy: {:.0%}'.format(total_correct/total))

    

0.6915871500968933
1.0390114784240723
1.9574806690216064
1.7543612718582153
0.6845536231994629
1.0137183666229248
0.8264726400375366
0.6764806509017944
0.6742739081382751
0.7091284990310669
0.8201663494110107
0.7052427530288696
0.7032874822616577
0.7058701515197754
0.7022926807403564
0.6695496439933777
0.7233829498291016
0.7077049016952515
0.7027021646499634
0.686415433883667
0.6998987793922424
0.6848814487457275
0.6758337020874023
0.6978969573974609
0.7021273970603943
0.6803317666053772
0.687629759311676
0.6858237981796265
0.6737041473388672
0.6928789019584656
0.6811394095420837
0.7050673961639404
0.6682025194168091
0.6759660243988037
0.6677019000053406
0.7132917642593384
0.6692762970924377
0.6785686612129211
0.6971730589866638
0.6520213484764099
0.6855332255363464
0.6697025299072266
0.6928859949111938
0.7090451121330261
0.6908478140830994
0.6799947619438171
0.6832920908927917
0.6722414493560791
0.6878982782363892
0.6927591562271118
Epoch:  0 , Loss:  0.754145964384079
Training accura

Training accuracy: 88%
Test accuracy: 74%
0.29023799300193787
0.2957703173160553
0.21815288066864014
0.3203911781311035
0.4109954833984375
0.28615522384643555
0.29489612579345703
0.2998989224433899
0.2777974307537079
0.38903212547302246
0.33640775084495544
0.3273164629936218
0.33553558588027954
0.29186761379241943
0.2555338740348816
0.2637486457824707
0.3530856966972351
0.21011993288993835
0.2779747247695923
0.29570117592811584
0.29375502467155457
0.40394216775894165
0.27368277311325073
0.37538814544677734
0.3407171666622162
0.30737611651420593
0.3050764501094818
0.31599724292755127
0.33928558230400085
0.40997904539108276
0.2594504654407501
0.2758556008338928
0.26744791865348816
0.2971087694168091
0.19979417324066162
0.3262222409248352
0.31175580620765686
0.3174740970134735
0.24521255493164062
0.29265451431274414
0.3161262571811676
0.3555377423763275
0.3603319525718689
0.34288719296455383
0.32132092118263245
0.36510229110717773
0.3631526231765747
0.315531462430954
0.3997563123703003
0.

0.21455368399620056
0.2740834355354309
0.28194114565849304
0.17101649940013885
0.23863202333450317
0.20043639838695526
0.2323230504989624
0.1492769867181778
0.07663100957870483
0.15478159487247467
0.19222235679626465
0.12686000764369965
Epoch:  15 , Loss:  0.1874751453101635
Training accuracy: 95%
Test accuracy: 74%
0.21557632088661194
0.20380406081676483
0.20362624526023865
0.11167893558740616
0.17940926551818848
0.13232775032520294
0.18194934725761414
0.12561379373073578
0.10239872336387634
0.08991820365190506
0.15175320208072662
0.1260189712047577
0.1284562051296234
0.12321250140666962
0.13289684057235718
0.131461039185524
0.13023824989795685
0.1354255974292755
0.2050091177225113
0.13111086189746857
0.19629232585430145
0.16270489990711212
0.1934753954410553
0.14222319424152374
0.11146748065948486
0.11498935520648956
0.10064157098531723
0.18288250267505646
0.2002207189798355
0.14114636182785034
0.251921147108078
0.12674978375434875
0.18579521775245667
0.19575752317905426
0.1141858845

0.08347001671791077
0.13705381751060486
0.2068844586610794
0.08554056286811829
0.10120834410190582
0.06842412799596786
0.06424069404602051
0.04868005961179733
0.08326072245836258
0.13018059730529785
0.1397329717874527
0.08071091771125793
0.1957976520061493
0.08654362708330154
0.0795007273554802
0.12649798393249512
0.054439444094896317
0.08764077723026276
0.11966076493263245
0.1635100394487381
0.058672185987234116
0.07194890081882477
0.17100004851818085
0.08032383024692535
0.0699554830789566
0.07304056733846664
0.10675084590911865
0.10143287479877472
0.0721098780632019
0.07353022694587708
Epoch:  23 , Loss:  0.0959632158279419
Training accuracy: 97%
Test accuracy: 74%
0.10720987617969513
0.1118527501821518
0.07294748723506927
0.07370191812515259
0.037194833159446716
0.06217729300260544
0.11264973878860474
0.025573663413524628
0.11737868189811707
0.07166926562786102
0.028696853667497635
0.052742380648851395
0.1561078429222107
0.1027589961886406
0.1369750201702118
0.08176209032535553
0.12

In [78]:
def text_inference(txt_inf):
    encoded = roberta.encode(txt_inf)
    encoded = torch.nn.functional.pad(encoded, pad=(0,400 - encoded.shape[0]), mode='constant', value=2)
    roberta.eval()
    model.eval()
    output = model(encoded).softmax(dim=1)
    return output


In [25]:
torch.save(model, 'model.pt')

In [48]:
torch.load('model.pt')

RoBERTaClassifier(
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=51200, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)